### English to Bahasa Translation using SeaLion-v3-9b-IT instruction tuned LLM based on Gemma2
### This is run in Runpod/remote Jupyter environment

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install
!git clone https://huggingface.co/Qwen/Qwen3-8B LLM_MODELS/Qwen3-8B

In [ ]:
import os
from pathlib import Path

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "workspace/LLM_MODELS/Qwen3-8B" ############## <--- Change this based on platform and models

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
######### set LLM_MODEL to the same path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization

print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

In [ ]:
!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
print("\nFinished translating dataset\n")